In [ ]:
from pathlib import Path

import zarr
from micro_sam.training import default_sam_loader
from ngio import open_ome_zarr_container


In [ ]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"


In [6]:
# Paths
zarr_path = Path(
    r"W:\groups\scratch\gmicro_prefect\ggrossha\ggrossha_SWI\training_data\accumulated_train.zarr"
)
cont = open_ome_zarr_container(zarr_path)
print(zarr.open(str(zarr_path), mode="r").tree())

/
 ├── 0 (3426, 1200, 1200) uint16
 ├── labels
 │   └── mask
 │       └── 0 (3426, 1200, 1200) uint32
 └── tables


In [10]:
raw_key = "0"
label_key = "labels/mask/0"
loader = default_sam_loader(
    raw_paths=str(zarr_path),
    raw_key=raw_key,
    label_paths=str(zarr_path),
    label_key=label_key,
    is_train=True,
    patch_shape=[2048, 2048],
    with_segmentation_decoder=True,
    batch_size=1,
)

In [11]:
# iterate over the loader to see if it works
for batch in loader:
    print(batch["raw"].shape, batch["labels"].shape)
    break

TypeError: list indices must be integers or slices, not str

In [17]:
for i in loader:
    print(len(i[0]))
    break

1
